In [85]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge lxml --yes 
from lxml import etree
#!conda install -c conda-forge geopy --yes 
import geopy.geocoders as gc
import requests 
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Read tables from URL: "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M".

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#### Explore how many tables are returned from URL. 

In [3]:
print("{} tables are returned from URL.".format(len(df)))
# from list of tables, find table which contains information about Toronto Naighborhoods, 
# and transform table to dataframe
df_toronto = pd.DataFrame(df[0]) 
df_toronto.head()

3 tables are returned from URL.


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Exclude row which contains 'Not assigned' value for Borough column 

In [4]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### From CSV file, read coordinates for each Borough. Set 'Postal Code' as index of data frame.

In [5]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_new = pd.DataFrame(df_coord)
df =df_new.set_index('Postal Code')

#### Set 'Postal Code' as index for data frame with information about Toronto neighborhood.

In [6]:
df_1 =df_toronto.set_index('Postal Code')

#### Merge both data frames into one data frame.

In [7]:
result = pd.concat([df_1, df], axis=1, sort=False)

#### Reset index of new data frame.

In [8]:
result.index.name = 'Postal Code'
result.reset_index(level = 'Postal Code')

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'XH5QPGKOHJ5J1DZO040CXLNR1ULCXPQYSPTVLA3RT1Z0XC4V' # your Foursquare ID
CLIENT_SECRET = 'FSZI1GFSNG3RVTNU4FMKH3QPLTJP5IBW0YAQYTD1YPBJDI53' # your Foursquare Secret
VERSION = '20200618' # Foursquare API version

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #if 'Toronto' in name:

        # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
        # make the GET request
            results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names=result['Neighborhood'],
                                 latitudes=result['Latitude'],
                                 longitudes=result['Longitude']
                                )
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [72]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 271 uniques categories.


In [73]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [74]:
toronto_grouped.drop(toronto_grouped.columns.difference(['Neighborhood','College Gym','Gym','Gym / Fitness Center']), axis=1, inplace=True)


In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Gym / Fitness Center,Gym,College Gym
1,"Alderwood, Long Branch",Gym,Gym / Fitness Center,College Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",Gym / Fitness Center,Gym,College Gym
3,Bayview Village,Gym / Fitness Center,Gym,College Gym
4,"Bedford Park, Lawrence Manor East",Gym / Fitness Center,Gym,College Gym


In [80]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 0, 0, 3, 2, 0], dtype=int32)

In [83]:
# add clustering labels
neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], axis=1, inplace= True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = result

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(axis=0, inplace=True)
toronto_merged.head(15) # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
Postal Code,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Gym / Fitness Center,Gym,College Gym
M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Gym / Fitness Center,Gym,College Gym
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Gym / Fitness Center,Gym,College Gym
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Gym / Fitness Center,Gym,College Gym
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Gym,Gym / Fitness Center,College Gym
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Gym / Fitness Center,Gym,College Gym
M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Gym / Fitness Center,College Gym
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2.0,Gym / Fitness Center,Gym,College Gym
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0.0,Gym / Fitness Center,Gym,College Gym


In [91]:
latitude = 43.651070
longitude = -79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype('int64')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
Postal Code,,,,
M7A,"Queen's Park, Ontario Provincial Government",Gym,Gym / Fitness Center,College Gym
M5C,St. James Town,Gym,Gym / Fitness Center,College Gym
M4G,Leaside,Gym,Gym / Fitness Center,College Gym
M4H,Thorncliffe Park,Gym,Gym / Fitness Center,College Gym
M5H,"Richmond, Adelaide, King",Gym,Gym / Fitness Center,College Gym
M6K,"Brockton, Parkdale Village, Exhibition Place",Gym,Gym / Fitness Center,College Gym
M4L,"India Bazaar, The Beaches West",Gym,Gym / Fitness Center,College Gym
M5L,"Commerce Court, Victoria Hotel",Gym,Gym / Fitness Center,College Gym
M6S,"Runnymede, Swansea",Gym,Gym / Fitness Center,College Gym
